In [9]:
!pip install -q langchain-groq langgraph>=0.0.40 langchain-core typing-extensions langchain


In [2]:
import os
os.environ["GROQ_API_KEY"] = "Your_Api_Key"

In [38]:
import re
from typing import Dict, Any, List
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [39]:
class AgentState(TypedDict):
    messages: List[Any]
    query_type: str
    result: str

# Custom Mathematical Functions
@tool
def plus(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtract the second number from the first number."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b

@tool
def divide(a: float, b: float) -> float:
    """Divide the first number by the second number."""
    if b == 0:
        raise ValueError("Cannot divide by zero!")
    return a / b

print("✅ Mathematical tools defined successfully!")
print("🔧 Available tools: plus, subtract, multiply, divide")

✅ Mathematical tools defined successfully!
🔧 Available tools: plus, subtract, multiply, divide


In [40]:
class MathematicalAgent:
    def __init__(self, api_key: str = None):
        """Initialize the Mathematical Agent with LLM and tools."""
        if api_key:
            os.environ["GROQ_API_KEY"] = api_key

        # Initialize LLM
        self.llm = ChatGroq(
            model="llama3-8b-8192",
            temperature=0.1,
            max_tokens=1000
        )

        # Define mathematical tools
        self.math_tools = [plus, subtract, multiply, divide]

        # Mathematical operation patterns
        self.math_patterns = {
            'addition': [
                r'(\d+(?:\.\d+)?)\s*\+\s*(\d+(?:\.\d+)?)',
                r'add\s+(\d+(?:\.\d+)?)\s+(?:and\s+|to\s+)?(\d+(?:\.\d+)?)',
                r'plus\s+(\d+(?:\.\d+)?)\s+(?:and\s+)?(\d+(?:\.\d+)?)',
                r'sum\s+of\s+(\d+(?:\.\d+)?)\s+and\s+(\d+(?:\.\d+)?)',
                r'what\s+is\s+(\d+(?:\.\d+)?)\s+plus\s+(\d+(?:\.\d+)?)',
                r'calculate\s+(\d+(?:\.\d+)?)\s*\+\s*(\d+(?:\.\d+)?)'
            ],
            'subtraction': [
                r'(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)',
                r'subtract\s+(\d+(?:\.\d+)?)\s+from\s+(\d+(?:\.\d+)?)',
                r'minus\s+(\d+(?:\.\d+)?)\s+(?:from\s+)?(\d+(?:\.\d+)?)',
                r'what\s+is\s+(\d+(?:\.\d+)?)\s+minus\s+(\d+(?:\.\d+)?)',
                r'calculate\s+(\d+(?:\.\d+)?)\s*-\s*(\d+(?:\.\d+)?)'
            ],
            'multiplication': [
                r'(\d+(?:\.\d+)?)\s*\*\s*(\d+(?:\.\d+)?)',
                r'multiply\s+(\d+(?:\.\d+)?)\s+(?:by\s+|and\s+)?(\d+(?:\.\d+)?)',
                r'times\s+(\d+(?:\.\d+)?)\s+(?:by\s+)?(\d+(?:\.\d+)?)',
                r'what\s+is\s+(\d+(?:\.\d+)?)\s+times\s+(\d+(?:\.\d+)?)',
                r'calculate\s+(\d+(?:\.\d+)?)\s*\*\s*(\d+(?:\.\d+)?)'
            ],
            'division': [
                r'(\d+(?:\.\d+)?)\s*/\s*(\d+(?:\.\d+)?)',
                r'divide\s+(\d+(?:\.\d+)?)\s+by\s+(\d+(?:\.\d+)?)',
                r'(\d+(?:\.\d+)?)\s+divided\s+by\s+(\d+(?:\.\d+)?)',
                r'what\s+is\s+(\d+(?:\.\d+)?)\s+divided\s+by\s+(\d+(?:\.\d+)?)',
                r'calculate\s+(\d+(?:\.\d+)?)\s*/\s*(\d+(?:\.\d+)?)'
            ]
        }

        # Build the graph
        self.graph = self._build_graph()

    def _detect_math_query(self, query: str) -> tuple:
        """Detect if query is mathematical and extract operation and numbers."""
        query_lower = query.lower().strip()

        for operation, patterns in self.math_patterns.items():
            for pattern in patterns:
                match = re.search(pattern, query_lower)
                if match:
                    try:
                        if operation == 'subtraction' and 'subtract' in pattern:
                            # For "subtract X from Y", we want Y - X
                            num1, num2 = float(match.group(2)), float(match.group(1))
                        else:
                            num1, num2 = float(match.group(1)), float(match.group(2))
                        return operation, num1, num2
                    except (ValueError, IndexError):
                        continue

        return None, None, None

    def _chatbot_node(self, state: AgentState) -> AgentState:
        """Handle general queries using LLM."""
        messages = state["messages"]

        # Create system message for the LLM
        system_msg = SystemMessage(content="""You are a helpful AI assistant.
        You can answer general questions about any topic. If you receive a mathematical
        question, just respond normally - the system will handle math operations separately.
        Keep your responses clear, helpful, and informative.""")

        # Get LLM response
        response = self.llm.invoke([system_msg] + messages)

        state["messages"].append(response)
        state["result"] = response.content
        state["query_type"] = "general"

        return state

    def _math_node(self, state: AgentState) -> AgentState:
        """Handle mathematical queries using custom functions."""
        last_message = state["messages"][-1].content
        operation, num1, num2 = self._detect_math_query(last_message)

        try:
            if operation == "addition":
                result = plus.invoke({"a": num1, "b": num2})
                answer = f"✅ The sum of {num1} and {num2} is {result}"
            elif operation == "subtraction":
                result = subtract.invoke({"a": num1, "b": num2})
                answer = f"✅ {num1} minus {num2} equals {result}"
            elif operation == "multiplication":
                result = multiply.invoke({"a": num1, "b": num2})
                answer = f"✅ {num1} times {num2} equals {result}"
            elif operation == "division":
                result = divide.invoke({"a": num1, "b": num2})
                answer = f"✅ {num1} divided by {num2} equals {result}"
            else:
                answer = "❌ I couldn't identify the mathematical operation."

        except Exception as e:
            answer = f"❌ Error performing calculation: {str(e)}"

        # Add AI response to messages
        ai_response = AIMessage(content=answer)
        state["messages"].append(ai_response)
        state["result"] = answer
        state["query_type"] = "mathematical"

        return state

    def _determine_route(self, state: AgentState) -> str:
        """Determine whether to route to math or chatbot node."""
        last_message = state["messages"][-1].content
        operation, _, _ = self._detect_math_query(last_message)

        if operation:
            return "math_node"
        else:
            return "chatbot_node"

    def _build_graph(self) -> StateGraph:
        """Build the LangGraph workflow."""
        workflow = StateGraph(AgentState)

        # Add nodes
        workflow.add_node("chatbot_node", self._chatbot_node)
        workflow.add_node("math_node", self._math_node)

        # Set the entry point with conditional routing
        workflow.set_conditional_entry_point(
            self._determine_route,
            {
                "chatbot_node": "chatbot_node",
                "math_node": "math_node"
            }
        )

        # Add edges from nodes to END
        workflow.add_edge("chatbot_node", END)
        workflow.add_edge("math_node", END)

        return workflow.compile()

    def query(self, user_input: str) -> str:
        """Process user query through the agent."""
        # Initialize state
        initial_state = {
            "messages": [HumanMessage(content=user_input)],
            "query_type": "",
            "result": ""
        }

        # Run the graph
        result = self.graph.invoke(initial_state)

        return result["result"]

print("✅ MathematicalAgent class defined successfully!")

✅ MathematicalAgent class defined successfully!


In [41]:
def run_colab_interactive_session(agent):
    """Run an interactive session optimized for Google Colab."""
    print("📚 Ask me general mathematical problems!")
    print("⏹️  Type 'quit' to exit\n")

    session_count = 0
    max_interactions = 10

    while session_count < max_interactions:
        try:
            user_input = input(f"[{session_count + 1}/{max_interactions}] You: ").strip()

            if user_input.lower() in ['quit', 'exit', 'bye', 'stop']:
                print("👋 Session ended!")
                break

            if not user_input:
                print("Please enter a question or mathematical problem.")
                continue

            print("🔄 Processing...")
            response = agent.query(user_input)
            print(f"🤖 Agent: {response}\n")

            session_count += 1

        except KeyboardInterrupt:
            print("\n👋 Session interrupted!")
            break
        except Exception as e:
            print(f"❌ Error: {str(e)}\n")

    if session_count >= max_interactions:
        print(f"📝 Session limit reached ({max_interactions} interactions)")
        print("💡 Restart the cell to continue chatting!")

print("✅ Interactive session function ready!")

✅ Interactive session function ready!


In [ ]:
run_colab_interactive_session(agent)

🤖 Mathematical Agent - Interactive Mode
📚 Ask me general questions or mathematical problems!
🔢 Math operations supported: +, -, *, /
⏹️  Type 'quit' to exit

[1/10] You: what is sum of one and 5
🔄 Processing...
🤖 Agent: The sum of 1 and 5 is 6.

[2/10] You: can you say the different between 10 and five 
🔄 Processing...
🤖 Agent: The difference between 10 and 5 is 5.

10 - 5 = 5

